# Importsand preparations

In [1]:
import pandas as pd
import torch
import os
import numpy as np
import datasets
import transformers
from GPUtil import showUtilization as gpu_usage
from numba import cuda
import torch.nn.functional as F

from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
)

from datasets import load_dataset, Dataset, DatasetDict

/itf-fi-ml/home/krimhau/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# !watch -n 0.5 nvidia-smi

In [3]:
print(f'PyTorch version: {torch.__version__}')  # 1.9.1+cu111
print(f'CUDA version: {torch.version.cuda}')  # 11.1
print(f'cuDNN version: {torch.backends.cudnn.version()}')  # 8005
print(f'Current device: {torch.cuda.current_device()}')  # 0
print(f'Is cuda available: {torch.cuda.is_available()}')  # TRUE

PyTorch version: 2.1.0+cu121
CUDA version: 12.1
cuDNN version: 8902
Current device: 0
Is cuda available: True


In [4]:
print(f'Transformers version: {transformers.__version__}')
print(f'Datasets version: {datasets.__version__}')

Transformers version: 4.35.0
Datasets version: 2.14.6


In [5]:
# Prevent a warning related to the tokenization process in the transformers library. 
os.environ["TOKENIZERS_PARALLELISM"] = "False"
# Makes CUDA operations synchronous
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [6]:
# Find the GPU with the least memory usage.
!nvidia-smi

Mon Nov 27 08:37:09 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2080 Ti      On | 00000000:01:00.0 Off |                  N/A |
| 22%   37C    P2               62W / 250W|    949MiB / 11264MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [7]:
def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    # free unreferenced tensors from the GPU memory.
    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

free_gpu_cache()   

Initial GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% |  8% |
|  1 |  0% |  0% |
|  2 |  0% |  0% |
|  3 |  0% |  0% |
|  4 |  0% | 12% |
|  5 |  0% |  0% |
|  6 |  0% |  0% |
|  7 |  0% |  0% |
GPU Usage after emptying the cache
| ID | GPU | MEM |
------------------
|  0 |  0% |  8% |
|  1 |  0% |  1% |
|  2 |  0% |  0% |
|  3 |  0% |  0% |
|  4 |  0% | 12% |
|  5 |  0% |  0% |
|  6 |  0% |  0% |
|  7 |  0% |  0% |


In [8]:
data = pd.read_csv("csv/clean_priority_high_or_not_high.csv" , index_col = 0)
data

,text_clean,label
0,autolog out after some time frame is your feat...,0
1,image picker for sourceimplement an android im...,0
2,fix video page listitem hovering behaviour whe...,0
3,escape shuttle reaches ludicrous speed descrip...,0
4,binder doesnt load notebooks outside of the un...,0
...,...,...
398144,running monomerbased executables on windowshi ...,1
398145,shamir 1ofx groups scheme where someeach group...,1
398146,setting fishhistory to empty in configfish cra...,1
398147,augment diverse datasetcreate script to augmen...,1


In [9]:
data_test_set = pd.read_csv("csv/testset_priority_high_or_not_high_clean.csv" , index_col = 0)
data_test_set

,text_clean,label
0,cardano indonesia meeting 3 24 january 2022 un...,1
1,expose active services rest api card the micro...,1
2,querycachememoryleakteststressuserlistenerremo...,1
3,fix linter errors for applicationnow that lint...,0
4,order show updated dateadd date updated to sho...,1
...,...,...
44234,comprehensive themingin the comprehensive them...,1
44235,post title is not appearing when a post is sha...,0
44236,checkeredflag emoji is missing from reactions...,0
44237,mapbox crashes randomly normally in low gps ar...,0


In [10]:
# Smaller and faster than bert.
base_model_id = "distilbert-base-uncased"

epochs = 5 #Number of full cyles through the training set.
num_labels = 2 #Number of labels, high, med, low priority.
learning_rate = 5e-5 # Rate the model updates based on the data its trained on.
train_batch_size = 16 # Number of training examples in one iteration.
eval_batch_size = 32 # Number evalutaion examples in on iteratoion.
save_strategy = "no" # Should the model be saved automatically during training.
save_steps = 500 # How often to save the model during training. No effect since no over.
logging_steps = 100
model_dir = "./model" #Where to save model

# Use early stopping to prevent overfitting
#load_best_model_at_end=True
#metric_for_best_model="eval_loss"
#greater_is_better=False

In [11]:

# Split dataframe into three parts: training, validation and testing.
def train_validate_test_split(df, train_percent=.8, validate_percent=.1, seed=42):
    np.random.seed(seed)
    # Shuffle index of dataframe
    perm = np.random.permutation(df.index)
    
    df_length = len(df.index)
    
    # Number of row in training set
    train_end = int(train_percent * df_length)
    # Number of rows in validate set
    validate_end = int(validate_percent * df_length) + train_end
    
    # From start to train end
    train = df.iloc[perm[:train_end]]
    # From train_end to validate_end
    validate = df.iloc[perm[train_end:validate_end]]
    # From validate to the last row in dataframe.
    test = df.iloc[perm[validate_end:]]
    return train, validate, test

In [12]:
# Drops rows with missing values
data.dropna(inplace=True)

In [13]:
# Resets the index after dropping rows
data.reset_index(inplace=True)
data

,index,text_clean,label
0,0,autolog out after some time frame is your feat...,0
1,1,image picker for sourceimplement an android im...,0
2,2,fix video page listitem hovering behaviour whe...,0
3,3,escape shuttle reaches ludicrous speed descrip...,0
4,4,binder doesnt load notebooks outside of the un...,0
...,...,...,...
398144,398144,running monomerbased executables on windowshi ...,1
398145,398145,shamir 1ofx groups scheme where someeach group...,1
398146,398146,setting fishhistory to empty in configfish cra...,1
398147,398147,augment diverse datasetcreate script to augmen...,1


In [14]:
#Drops the index col, better for managint the data.
data.drop(columns= ["index"], inplace = True)

In [15]:
data

,text_clean,label
0,autolog out after some time frame is your feat...,0
1,image picker for sourceimplement an android im...,0
2,fix video page listitem hovering behaviour whe...,0
3,escape shuttle reaches ludicrous speed descrip...,0
4,binder doesnt load notebooks outside of the un...,0
...,...,...
398144,running monomerbased executables on windowshi ...,1
398145,shamir 1ofx groups scheme where someeach group...,1
398146,setting fishhistory to empty in configfish cra...,1
398147,augment diverse datasetcreate script to augmen...,1


In [16]:
# 80% trainig, 10% validate, 10% test. Seed 42.
# Test 80-10-10 and 70-15-15
train , validate , test = train_validate_test_split(data)

In [17]:
train.set_index("label" , inplace = True)
validate.set_index("label" , inplace = True)
test.set_index("label" , inplace = True)

In [18]:
test

,text_clean
label,
0,drop appbotx in favor of ios 103 skstorereview...
0,selection support isgroupselected and passing ...
1,twitter report app shows begegnungen an 2 tage...
0,ui nodeview enhancementsthis is a meta thread ...
1,change login action to use public key descript...
...,...
1,65f15e2d8573446b8677summary test id tes1016050...
1,default buttonsbuttons should be pressed by de...
0,statistics add new filter on kw and timethis f...


In [19]:
# Convert from Pandas DataFrame to Hugging Face datasets
tds = Dataset.from_pandas(train)
vds = Dataset.from_pandas(validate)
testds = Dataset.from_pandas(test)

separate_test_set = Dataset.from_pandas(data_test_set)
ds = DatasetDict()

ds["test"] = testds
ds["train"] = tds
ds["validate"] = vds
ds["separate_test_set"] = separate_test_set

ds

DatasetDict({
    test: Dataset({
        features: ['text_clean', 'label'],
        num_rows: 39816
    })
    train: Dataset({
        features: ['text_clean', 'label'],
        num_rows: 318519
    })
    validate: Dataset({
        features: ['text_clean', 'label'],
        num_rows: 39814
    })
    separate_test_set: Dataset({
        features: ['text_clean', 'label', '__index_level_0__'],
        num_rows: 44239
    })
})

In [20]:
train_dataset = ds["train"]
valid_dataset = ds["validate"]
test_ds = ds["test"]
separate_test_set_dataset = ds["separate_test_set"]

In [21]:
ds["train"][0]

{'text_clean': 'prevent deletions and insertions from duplication when movingcopying strandscreate two strands bound on a helix and add a deletion imagehttpsuserimagesgithubusercontentcom1927436592946017f2e27880f40a11ea9f7c07a8eef2b581png create a new strand elsewhere imagehttpsuserimagesgithubusercontentcom192743659294608005f54880f40b11ea8f92d58997f07bbbpng move the bottom strand to be bound to the new strand note that one has a deletion and the other does not imagehttpsuserimagesgithubusercontentcom1927436592946138160d2800f40b11ea99066103ddd7de3epng add a deletion at the same offset by clicking on the strand lacking the deletion imagehttpsuserimagesgithubusercontentcom192743659294618626250780f40b11ea9075ddc6a17115f6png now the bottom strand has two deletions stored in the scadnano file json version 0120 grid square helices gridposition 0 0 maxoffset 64 gridposition 0 1 maxoffset 64 strands color cc0000 domains helix 0 forward true start 0 end 8 deletions 3 color 32b86c domains helix 

In [22]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForSequenceClassification.from_pretrained(base_model_id, num_labels=num_labels)
#tokenizer = AutoTokenizer.from_pretrained(base_model_id)
# optim = torch.optim.Adam(model.parameters(), lr=5e-5)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'classifier.weight', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Tokenization

In [24]:
model_ckpt = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

    Tokenizing the whole dataset

In [25]:
#Tokenize the dataset to the correct input for the transformer model.
def tokenize(batch):
    return tokenizer(batch["text_clean"], padding="max_length", truncation=True)

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
valid_dataset = valid_dataset.map(tokenize, batched=True, batch_size=len(valid_dataset))
test_dataset = test_ds.map(tokenize, batched=True, batch_size=len(test_ds))
separate_test_set_dataset = separate_test_set_dataset.map(tokenize, batched=True, batch_size=len(separate_test_set_dataset))

Map: 100%|████████████████████████████████████████| 44239/44239 [00:37<00:00, 1184.60 examples/s]


## Training a classifier

In [26]:
training_args = TrainingArguments(
    output_dir=model_dir,
    num_train_epochs=epochs,
    per_device_train_batch_size=train_batch_size,
    per_device_eval_batch_size=eval_batch_size,
    save_strategy=save_strategy,
    save_steps=save_steps,
    evaluation_strategy="epoch",
    learning_rate=learning_rate,
    logging_steps=logging_steps,
)

In [27]:
 trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
)

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [28]:
trainer.train() 

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.618700,0.612835,0.679861,[0.6876899 0.67163026],[0.69083838 0.66842726],[0.68456998 0.6748641 ]
2,0.562800,0.562163,0.694203,[0.69839225 0.68989583],[0.70948259 0.67898325],[0.6876433 0.7011649]
3,0.520600,0.566718,0.704350,[0.70962331 0.6988821 ],[0.71778621 0.69073625],[0.70164398 0.70722237]
4,0.442600,0.625053,0.705280,[0.71813596 0.69119427],[0.70739672 0.70288498],[0.7292063 0.6798861]
5,0.373200,0.685342,0.701562,[0.71682555 0.68455984],[0.70075952 0.70250095],[0.73364554 0.6675123 ]


TrainOutput(global_step=99540, training_loss=0.499676262616201, metrics={'train_runtime': 50815.3335, 'train_samples_per_second': 31.341, 'train_steps_per_second': 1.959, 'total_flos': 2.1096691676255232e+17, 'train_loss': 0.499676262616201, 'epoch': 5.0})

* Training loss: Difference between the predictons made by the model on the training dataset vs on the actual data.
* Validation loss: how well the model functions on unseen data.
* Accuracy: How much the model gets correct. number of correct Prediction / total number of predictions.
* F1: consider both precision and recall. 
* Precision: Accuracy of positive predictions. Percison TP = TP + FP. How often the model is correct.
* Recall: True positive rate. how many items the model gets correct from the total amount.

### Training loss decreases, valdiation loss increases = Overfitting

In [29]:
# Evaluate validation set
eval_result = trainer.evaluate(eval_dataset=valid_dataset)

In [30]:
for key, value in sorted(eval_result.items()):
    print(f"{key} = {value}\n")

epoch = 5.0

eval_accuracy = 0.7015622645300648

eval_f1 = [0.71682555 0.68455984]

eval_loss = 0.6853417158126831

eval_precision = [0.70075952 0.70250095]

eval_recall = [0.73364554 0.6675123 ]

eval_runtime = 324.5009

eval_samples_per_second = 122.693

eval_steps_per_second = 3.837



In [31]:
# Evaluate test data set
test_results = trainer.evaluate(eval_dataset=test_dataset)

In [32]:
for key, value in sorted(test_results.items()):
    print(f"{key} = {value}\n")

epoch = 5.0

eval_accuracy = 0.7015772553747237

eval_f1 = [0.71714911 0.68419094]

eval_loss = 0.6853575110435486

eval_precision = [0.69927116 0.70429549]

eval_recall = [0.73596521 0.66520234]

eval_runtime = 333.8954

eval_samples_per_second = 119.247

eval_steps_per_second = 3.729



# Test set

In [33]:
separate_test_set_results = trainer.evaluate(eval_dataset=separate_test_set_dataset)

In [34]:
for key, value in sorted(separate_test_set_results.items()):
    print(f"{key} = {value}\n")

epoch = 5.0

eval_accuracy = 0.6958339926309365

eval_f1 = [0.71064855 0.67942059]

eval_loss = 0.6976377367973328

eval_precision = [0.69560093 0.69610428]

eval_recall = [0.7263616  0.66351792]

eval_runtime = 369.9535

eval_samples_per_second = 119.58

eval_steps_per_second = 3.738



In [35]:
trainer.save_model(model_dir + "_local") 

In [36]:
from transformers import pipeline
    
classifier = pipeline("text-classification", model="./model_local")

In [37]:
classifier.model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [38]:
classifier("this does not need to be done fast")

[{'label': 'LABEL_0', 'score': 0.5389364957809448}]

In [39]:
classifier("this is super important")

[{'label': 'LABEL_0', 'score': 0.9508143663406372}]

In [40]:
classifier("this bug has super high impact on the project")

[{'label': 'LABEL_0', 'score': 0.9637181162834167}]

## Important to delete large objects to free memory 
del train_dataset

In [41]:
del valid_dataset

In [42]:
del model

In [43]:
# Free cache
torch.cuda.empty_cache()

In [44]:
!nvidia-smi

Mon Nov 27 23:08:13 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2080 Ti      On | 00000000:01:00.0 Off |                  N/A |
| 25%   45C    P2               63W / 250W|    949MiB / 11264MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--